In [1]:
DATA_NAME = 'syntheticMRI3D-full' 
TRANSFORM = 'wavelet'
CHANNEL = 'aad'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_aad_wavelet_full_syntheticMRI3D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) <100:
            X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,52844499.87,-1141.49,18.85,-26.31,-12.15,2.92
4,0.0,-1141.49,7100298.03,-25.89,20.61,10.92,1.87
5,0.0,18.85,-25.89,1212958.22,1.07,0.17,1.00
6,0.0,-26.31,20.61,1.07,157680.29,0.07,0.38
7,0.0,-12.15,10.92,0.17,0.07,13789.80,-0.17
8,0.0,2.92,1.87,1.00,0.38,-0.17,487.60


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_20685/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,-0.00006,0.00000,-0.00001,-0.00001,0.00002
4,NaN,-0.00006,1.00000,-0.00001,0.00002,0.00003,0.00003
5,NaN,0.00000,-0.00001,1.00000,0.00000,0.00000,0.00004
6,NaN,-0.00001,0.00002,0.00000,1.00000,0.00000,0.00004
7,NaN,-0.00001,0.00003,0.00000,0.00000,1.00000,-0.00007
8,NaN,0.00002,0.00003,0.00004,0.00004,-0.00007,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1615.8125759147254

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.99147337e+14 7.00693566e+12 1.96143107e+11 3.10713258e+09
 2.11172681e+07 1.98060303e+04 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.0,0.999732,-0.022810,-0.003812,-0.000496,-0.000044,-0.000001
1,0.0,0.022659,0.999113,-0.035228,-0.004418,-0.000385,-0.000013
2,0.0,0.004590,0.034965,0.998823,-0.033182,-0.002841,-0.000100
3,-0.0,0.000746,0.005550,0.032907,0.999008,-0.029450,-0.001028
4,0.0,0.000087,0.000646,0.003792,0.029315,0.999405,-0.017744
5,-0.0,0.000005,0.000034,0.000200,0.001544,0.017706,0.999842
6,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.9773408313334178,
 0.9650346174967023,
 0.967093388479032,
 0.9706849451160968,
 0.982293897707728,
 1.0]